**Note**: 
This notebook contains code blocks for Optuna hyperparameter tuning across various base and meta models. It’s not meant to be run as-is—mainly because the features used (wrangle() function) here is based on earlier versions of the dataset and features. The current feature engineering pipeline (in the main notebook) has evolved, and so the scores and results in this notebook may differ.

### Base Model Trials

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.linear_model import Ridge, Lasso, ElasticNet, HuberRegressor, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, StackingRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer
import optuna
from optuna import create_study
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from utils.data_prep import load_and_prepare_data
from utils.pipeline_create import create_pipeline

In [ ]:
data = load_and_prepare_data()

X_train = data['X_train']
X_val1 = data['X_val1']
X_val2 = data['X_val2']
y_train = data['y_train']
y_val1 = data['y_val1']
y_val2 = data['y_val2']

features = data['features']

There were several rounds of hyperparameter tuning using Optuna. After each study, valuable insights were gathered and used to guide future feature engineering, model selection, and subsequent tuning strategies.

### First set of optuna tuning

In [ ]:
def objective(trial):
    params = {
        'alpha': trial.suggest_float('alpha', 1e-4, 1e4, log=True),
        'solver': trial.suggest_categorical('solver', ['auto', 'svd', 'cholesky', 'lsqr', 'sag', 'saga']),
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'tol': trial.suggest_float('tol', 1e-4, 1e-2, log=True), 
    }

    pipeline = create_pipeline('ridge', Ridge, params, features)
    
    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    return score.mean()

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42),  pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5))
study.optimize(objective, n_trials=100, n_jobs=-1)

In [ ]:
def objective(trial):
    params = {
        'alpha': trial.suggest_float('alpha', 1e-4, 1e4, log=True),
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'tol': trial.suggest_float('tol', 1e-4, 1e-2, log=True),
        'max_iter': trial.suggest_int('max_iter', 1000, 10000),
        'selection': trial.suggest_categorical('selection', ['cyclic', 'random'])
    }

    pipeline = create_pipeline('lasso', Lasso, params, features)
    
    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    return score.mean()

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42),  pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5))
study.optimize(objective, n_trials=100, n_jobs=-1)

In [ ]:
def objective(trial):
    params = {
        'alpha': trial.suggest_float('alpha', 1e-4, 1e2, log=True),
        'l1_ratio': trial.suggest_float('l1_ratio', 0.1, 0.9),
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'tol': trial.suggest_float('tol', 1e-4, 1e-2, log=True),
        'max_iter': trial.suggest_int('max_iter', 5000, 10000),
        'selection': trial.suggest_categorical('selection', ['cyclic', 'random'])
    }
  
    pipeline = create_pipeline('elasticnet', ElasticNet, params, features)

    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    return score.mean()
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42),  pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5))
study.optimize(objective, n_trials=100, n_jobs=-1)

In [ ]:
def objective(trial):
    params = {
        'alpha_1': trial.suggest_float('alpha_1', 1e-7, 1e-1, log=True),
        'alpha_2': trial.suggest_float('alpha_2', 1e-7, 1e-1, log=True),
        'lambda_1': trial.suggest_float('lambda_1', 1e-7, 1e-1, log=True),
        'lambda_2': trial.suggest_float('lambda_2', 1e-7, 1e-1, log=True),
        'tol': trial.suggest_float('tol', 1e-4, 1e-2, log=True),
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'compute_score': trial.suggest_categorical('compute_score', [True, False])
    }

    pipeline = create_pipeline('bayesianridge', BayesianRidge, params, features)
    
    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    return score.mean()
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42),  pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5))
study.optimize(objective, n_trials=100, n_jobs=-1)

In [ ]:
def objective(trial):
    params = {
        'epsilon': trial.suggest_float('epsilon', 1.0, 2.0),
        'alpha': trial.suggest_float('alpha', 1e-5, 1e1, log=True),
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'max_iter': trial.suggest_int('max_iter', 1000, 2000),
        'tol': trial.suggest_float('tol', 1e-4, 1e2, log=True)
    }

    pipeline = create_pipeline('huberregressor', HuberRegressor, params, features)
    
    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')    
    return score.mean()
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42),  pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5))
study.optimize(objective, n_trials=100, n_jobs=-1)

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 10, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 4),
        'max_features': trial.suggest_categorical('max_features', [None, 'sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [False])
    }

    pipeline = create_pipeline('randomforest', RandomForestRegressor, params, features)
    
    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    return score.mean()
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42),  pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5))
study.optimize(objective, n_trials=50, n_jobs=1)

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 10, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 4),
        'max_features': trial.suggest_categorical('max_features', [None, 'sqrt', 'log2']),
        'bootstrap': trial.suggest_categorical('bootstrap', [False])
    }

    pipeline = create_pipeline('extratrees', ExtraTreesRegressor, params, features)
    
    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    return score.mean()
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42),  pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5))
study.optimize(objective, n_trials=50, n_jobs=1)

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 4),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    }
  
    pipeline = create_pipeline('gradientboosting', GradientBoostingRegressor, params, features)
    
    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    return score.mean()
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42),  pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5))
study.optimize(objective, n_trials=50, n_jobs=1)

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 1e2, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 1e2, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'booster': trial.suggest_categorical('booster', ['gbtree']),
    }

    pipeline = create_pipeline('xgboost', XGBRegressor, params, features)
    
    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    return score.mean()
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42),  pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5))
study.optimize(objective, n_trials=50, n_jobs=1)

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 15, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 1e2, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 1e2, log=True),
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt']),
    }

    pipeline = create_pipeline('lightgbm', LGBMRegressor, params, features)

    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    return score.mean()
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42),  pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5))
study.optimize(objective, n_trials=50, n_jobs=1)

In [ ]:
def objective(trial):
    bootstrap_type = trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli'])

    params = {
        'iterations': trial.suggest_int('iterations', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 32, 128),
        'random_strength': trial.suggest_float('random_strength', 1, 10),
        'bootstrap_type': bootstrap_type,
        'grow_policy': trial.suggest_categorical('grow_policy', ['Depthwise', 'Lossguide']),
        }

    # Only add bagging_temperature if bootstrap_type is Bayesian
    if bootstrap_type == 'Bayesian':
        params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 1)

    pipeline = create_pipeline('catboost', CatBoostRegressor, params, features)
    
    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    return score.mean()
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42),  pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5))
study.optimize(objective, n_trials=50, n_jobs=1)

### Best Parameters for models

<details>
<summary><strong>Linear Models</strong></summary>

| Model         | CV Score (RMSE)    | Best Parameters |
|---------------|----------|-----------------|
| Ridge         | 0.17207  | {'alpha': 0.4402, 'solver': 'sag', 'fit_intercept': True, 'tol': 0.000562} |
| Lasso         | 0.17208  | {'alpha': 1.01e-4, 'fit_intercept': True, 'tol': 0.000118, 'max_iter': 6191, 'selection': 'cyclic'} |
| ElasticNet    | 0.17207  | {'alpha': 1.01e-4, 'l1_ratio': 0.1083, 'fit_intercept': True, 'tol': 0.00215, 'max_iter': 5473, 'selection': 'cyclic'} |
| BayesianRidge | 0.17207  | {'alpha_1': 1.51e-4, 'alpha_2': 0.0257, 'lambda_1': 0.00117, 'lambda_2': 0.0995, 'tol': 0.00373, 'fit_intercept': True, 'compute_score': False} |
| Huber         | 0.17258  | {'epsilon': 1.9999, 'alpha': 0.00074, 'fit_intercept': True, 'max_iter': 1911, 'tol': 0.02199} |

</details>

<details>
<summary><strong>Tree-Based Models</strong></summary>

| Model           | CV Score (RMSE)    | Best Parameters |
|----------------|----------|-----------------|
| RandomForest    | 0.06202  | {'n_estimators': 376, 'max_depth': 25, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'bootstrap': False} |
| ExtraTrees      | 0.06066  | {'n_estimators': 444, 'max_depth': 18, 'min_samples_split': 9, 'min_samples_leaf': 1, 'max_features': None, 'bootstrap': False} |

</details>

<details>
<summary><strong>Boosting Models</strong></summary>

| Model             | CV Score (RMSE)     | Best Parameters |
|------------------|----------|-----------------|
| GradientBoosting | 0.05971  | {'n_estimators': 492, 'learning_rate': 0.0165, 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 3, 'subsample': 0.8637, 'max_features': None} |
| XGBoost          | 0.06020  | {'n_estimators': 166, 'learning_rate': 0.0454, 'max_depth': 8, 'subsample': 0.9997, 'colsample_bytree': 0.6015, 'reg_alpha': 0.6610, 'reg_lambda': 0.00162, 'min_child_weight': 5, 'gamma': 0.1865, 'booster': 'gbtree'} |
| LightGBM         | 0.05968  | {'n_estimators': 360, 'learning_rate': 0.0690, 'max_depth': 10, 'num_leaves': 80, 'min_child_samples': 39, 'subsample': 0.6166, 'colsample_bytree': 0.7239, 'reg_alpha': 0.1303, 'reg_lambda': 0.3414, 'boosting_type': 'gbdt'} |
| CatBoost         | 0.06019  | {'bootstrap_type': 'Bayesian', 'iterations': 451, 'learning_rate': 0.1059, 'depth': 7, 'l2_leaf_reg': 9.9215, 'border_count': 97, 'random_strength': 1.0693, 'grow_policy': 'Lossguide', 'bagging_temperature': 0.0467} |

</details>

These trials helped in identifying the optimum parameter space for each model.  Using these best-found parameters, each model was then evaluated on a separate validation set  (see: `model_selection_validation.ipynb`) to select the final set of models to be used as base and meta models in the ensemble.

After this, Optuna tuning was done iteratively for the selected models — Ridge, ExtraTrees, XGBoost, LightGBM, and CatBoost. The newly engineered features, which were inspired by the permutation importance method from the ModelEvaluator, played a crucial role in creating interactions that helped improve the cross-validation scores of the models.

Different meta-models were explored with the passthrough parameter of StackingRegressor set to both True and False.
The trials and results are summarized below in tables.

The meta-model trial with an initial set of parameter values is also provided, which was later updated after several rounds of feature engineering and base model tuning.

### Meta Model trials

In [ ]:
def objective(trial):

    params = {
    'n_estimators': trial.suggest_int('n_estimators', 300, 1000),  
    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),  
    'max_depth': trial.suggest_int('max_depth', 2, 7), 
    'subsample': trial.suggest_float('subsample', 0.2, 1.0),
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
    'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),  
    'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
    'min_child_weight': trial.suggest_int('min_child_weight', 1, 6), 
    'gamma': trial.suggest_float('gamma', 0, 2),  
    'booster': trial.suggest_categorical('booster', ['gbtree'])
    }

    def drop_categorical_cols(X): # Different verisons of this function was used - with all features and with selected features 
        cols = [f'meta_{i}' for i in range(4)] + list(X_train.columns)
        df_X = pd.DataFrame(X, columns=cols)
        # selected_features = [f'meta_{i}' for i in range(4)] + ['cardio_load', 'heart_rate', 'row_mean', 'temp_duration_product', 'sex_male']
        # df_X = df_X[selected_features]
        return df_X.values

    meta_model = Pipeline(steps=
        [('drop_cat', FunctionTransformer(drop_categorical_cols, validate=False)),
        ('xgbregressor', XGBRegressor(**params))] 
    )
    # meta_model = XGBRegressor(**params)  # This is used when passthrough=False in StackingRegressor

    stack = StackingRegressor(
    estimators=[
        ('ridge', create_pipeline('ridge', Ridge, {'alpha': 0.4402, 'solver': 'sag', 'fit_intercept': True, 'tol': 0.000562}, features)),
        ('extratrees', create_pipeline('extratrees', ExtraTreesRegressor, {'n_estimators': 444, 'max_depth': 18, 'min_samples_split': 9, 'min_samples_leaf': 1, 'max_features': None, 'bootstrap': False}, features)),
        # ('xgboost', create_pipeline('xgboost', XGBRegressor, )
        ('catboost', create_pipeline('catboost', CatBoostRegressor, {'bootstrap_type': 'Bayesian', 'iterations': 451, 'learning_rate': 0.1059, 'depth': 7, 'l2_leaf_reg': 9.9215, 'border_count': 97, 'random_strength': 1.0693, 'grow_policy': 'Lossguide', 'bagging_temperature': 0.0467}, features)),
        ('lightgbm', create_pipeline('lightgbm', LGBMRegressor, {'n_estimators': 360, 'learning_rate': 0.0690, 'max_depth': 10, 'num_leaves': 80, 'min_child_samples': 39, 'subsample': 0.6166, 'colsample_bytree': 0.7239, 'reg_alpha': 0.1303, 'reg_lambda': 0.3414, 'boosting_type': 'gbdt'} , features)),
    ],
    final_estimator=meta_model,  
    passthrough=True,
    cv=KFold(n_splits=5, shuffle=True, random_state=42),
    n_jobs=1
)

    score = cross_val_score(stack, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    return score.mean()

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42), pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5))
study.optimize(objective, n_trials=100, n_jobs=1)

In [ ]:
def objective(trial):
    
    params = {
    'alpha': trial.suggest_float('alpha', 5e-4, 0.005, log=True),
    'l1_ratio': trial.suggest_float('l1_ratio', 0.6, 0.8),
    'fit_intercept': trial.suggest_categorical('fit_intercept', [True]),  
    'tol': trial.suggest_float('tol', 1e-4, 5e-4, log=True),
    'max_iter': trial.suggest_int('max_iter', 7000, 9500),
    'selection': trial.suggest_categorical('selection', ['cyclic']) 
}

    meta_model  = ElasticNet(**params)

    stack = StackingRegressor(
    estimators=[
        ('ridge', create_pipeline('ridge', Ridge, {'alpha': 0.3744806562337833, 'solver': 'sag', 'fit_intercept': True, 'tol': 0.0007622314553132402}, features)),
        ('extratrees', create_pipeline('extratrees', ExtraTreesRegressor, {'n_estimators': 326, 'max_depth': 24, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': False}, features)),
        ('xgboost', create_pipeline('xgboost', XGBRegressor, {'n_estimators': 683, 'learning_rate': 0.05710400944032593, 'max_depth': 7, 'subsample': 0.8708501983892822, 'colsample_bytree': 0.5990353703327878, 'reg_alpha': 3.254066913534751, 'reg_lambda': 0.01593621652838458, 'min_child_weight': 2, 'gamma': 0.0009536580000644827, 'booster': 'gbtree'}, features)),
        ('lightgbm', create_pipeline('lightgbm', LGBMRegressor, {'max_depth': 12, 'num_leaves': 99, 'n_estimators': 491, 'learning_rate': 0.05954360974397409, 'min_child_samples': 21, 'subsample': 0.5066852407414704, 'colsample_bytree': 0.8619994805735846, 'reg_alpha': 3.595735178870403, 'reg_lambda': 1.24462626575653}, features)),
        ('catboost', create_pipeline('catboost', CatBoostRegressor, {'bootstrap_type': 'Bernoulli', 'iterations': 406, 'learning_rate': 0.028763633853386924, 'depth': 10, 'l2_leaf_reg': 11.869905054427921, 'border_count': 125, 'random_strength': 7.610826130799793, 'grow_policy': 'Depthwise'}, features)),
    ],
    final_estimator=meta_model,  
    passthrough=False,
    cv=KFold(n_splits=5, shuffle=True, random_state=42),
    n_jobs=1
)

    score = cross_val_score(stack, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
    return score.mean()

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42), pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5))
study.optimize(objective, n_trials=100, n_jobs=1)


After the second round of feature engineering, the cross-validation scores and corresponding parameters for each model are shown in the table below.

| Model               | CV Score (RMSE)          | Best Parameters |
|---------------------|--------------------|-----------------|
| Ridge         | 0.1047889656803068 | {'alpha': 0.000261432296719797, 'solver': 'saga', 'fit_intercept': True, 'tol': 0.00045083197355793724} |
| ExtraTrees    | 0.0599177958540447 | {'n_estimators': 263, 'max_depth': 26, 'min_samples_split': 11, 'min_samples_leaf': 1, 'max_features': 'sqrt'} |
| LightGBM      | 0.05959388334439157 | {'max_depth': 10, 'num_leaves': 106, 'n_estimators': 421, 'learning_rate': 0.04405636101210134, 'min_child_samples': 21, 'subsample': 0.7534719178477073, 'colsample_bytree': 0.818906983809868, 'reg_alpha': 1.4268427054386101, 'reg_lambda': 0.036317188692636046} |
| CatBoost      | 0.05988123878061226 | {'bootstrap_type': 'Bayesian', 'iterations': 488, 'learning_rate': 0.0632052789546276, 'depth': 9, 'l2_leaf_reg': 13.183208159587302, 'border_count': 97, 'random_strength': 0.8206642961816146, 'grow_policy': 'Depthwise', 'bagging_temperature': 0.06947485130639368} |
| XGBoost (meta)  | 0.05977764678139798 | {'n_estimators': 580, 'learning_rate': 0.06960165632995971, 'max_depth': 6, 'subsample': 0.6429851993135641, 'colsample_bytree': 0.8790059462140852, 'reg_alpha': 2.7048951291792394, 'reg_lambda': 0.5262497975347594, 'min_child_weight': 9, 'gamma': 0.0046972338244777095} |

After residual analysis, new features were added and so hyperparameters were tuned again. The results are shown here in the table. Various combinations were tried for base and meta model combinations, and the best performing one was:

**Base models** → Ridge, ExtraTrees, XGBRegressor, LGBMRegressor, CatBoostRegressor  
**Meta model** → ElasticNet

This trial is shown in the last code block of this notebook.

The resulting parameters for base models and the meta model are shown in the table below — these are the finalized params.

| Model       | CV Score (RMSE) | Best Parameters |
|-------------|----------|-----------------|
| **Ridge**   | 0.07979994431657829 | {'alpha': 0.3744806562337833, 'solver': 'sag', 'fit_intercept': True, 'tol': 0.0007622314553132402} |
| **ExtraTrees** | 0.06000782291510082 | {'n_estimators': 326, 'max_depth': 24, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': False} |
| **XGBoost** | 0.05953003283657898 | {'n_estimators': 683, 'learning_rate': 0.05710400944032593, 'max_depth': 7, 'subsample': 0.8708501983892822, 'colsample_bytree': 0.5990353703327878, 'reg_alpha': 3.254066913534751, 'reg_lambda': 0.01593621652838458, 'min_child_weight': 2, 'gamma': 0.0009536580000644827, 'booster': 'gbtree'} |
| **LightGBM** | 0.0595578252334081 | {'max_depth': 12, 'num_leaves': 99, 'n_estimators': 491, 'learning_rate': 0.05954360974397409, 'min_child_samples': 21, 'subsample': 0.5066852407414704, 'colsample_bytree': 0.8619994805735846, 'reg_alpha': 3.595735178870403, 'reg_lambda': 1.24462626575653} |
| **CatBoost** | 0.05966427275867594 | {'bootstrap_type': 'Bernoulli', 'iterations': 406, 'learning_rate': 0.028763633853386924, 'depth': 10, 'l2_leaf_reg': 11.869905054427921, 'border_count': 125, 'random_strength': 7.610826130799793, 'grow_policy': 'Depthwise'} |
| **ElasticNet (Meta)** | 0.0590764132136256 | {'alpha': 0.0007174822830545591, 'l1_ratio': 0.658029960276858, 'fit_intercept': True, 'tol': 0.00010062095710653478, 'max_iter': 8792, 'selection': 'cyclic'} |